In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import requests

https://developers.google.com/sheets/api/quickstart/python

In [2]:
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("../keys/token.json"):
    creds = Credentials.from_authorized_user_file("../keys/token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "../keys/credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=0)
    # flow = InstalledAppFlow.from_client_secrets_file(
    #     "../keys/credentials.json", SCOPES
    # )
    # creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("../keys/token.json", "w") as token:
        token.write(creds.to_json())

In [3]:
# SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]
# creds = Credentials.from_authorized_user_file("../keys/token.json", SCOPES)
spreadsheetId = "16f8IvbY7bcCBA8X2xTbgzP2Mscuol9SgpVml1XTkNbQ" # Please set the Spreadsheet ID.

sheets = build("sheets", "v4", credentials=creds)

sheetObj = sheets.spreadsheets().get(spreadsheetId=spreadsheetId, fields="sheets(properties(sheetId,title))").execute()
accessToken = creds.token
for s in sheetObj.get("sheets", []):
    p = s["properties"]
    sheetName = p["title"]
    if sheetName != 'Past Trades':
        continue
    print("Download: " + sheetName)
    url = "https://docs.google.com/spreadsheets/export?id=" + spreadsheetId + "&exportFormat=csv&gid=" + str(p["sheetId"])
    res = requests.get(url, headers={"Authorization": "Bearer " + accessToken})
    with open('../data/' + sheetName + ".csv", mode="wb") as f:
        f.write(res.content)

Download: Past Trades


In [ ]:
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms"
SAMPLE_RANGE_NAME = "Class Data!A2:E"
def main():
  """Shows basic usage of the Sheets API.
  Prints values from a sample spreadsheet.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("sheets", "v4", credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = (
        sheet.values()
        .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
        .execute()
    )
    values = result.get("values", [])

    if not values:
      print("No data found.")
      return

    print("Name, Major:")
    for row in values:
      # Print columns A and E, which correspond to indices 0 and 4.
      print(f"{row[0]}, {row[4]}")
  except HttpError as err:
    print(err)
main()